In [1]:
# Copyright 2021-2022 Boris Shminke
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
# saturation prover environment can be created
# as any other OpenAI Gym environment
import gym
import os
from glob import glob
import sys
# this example uses package's resources
# one can indicate the path to unpacked (TPTP)[http://tptp.org/] instead
# it's obligatory to set the list of problems
# Here are CNF problems from TPTP
if sys.version_info.major == 3 and sys.version_info.minor >= 9:
    from importlib.resources import files
else:
    from importlib_resources import files

problem_list = sorted(glob(
    os.path.join(
        files(
            "gym_saturation"
        ).joinpath(os.path.join(
            "resources", "TPTP-mock"
        ),
        os.path.join("Problems", "*", "*-*.p")
    )
)))
env = gym.make(
    "GymSaturation-v0",
    problem_list=problem_list,
    disable_env_checker=True
)

In [3]:
# to make results reproducible, let's set the seed
print(env.seed(2))

2


In [4]:
# parsing is done with `lark` package which sometimes might be slow
# reimplementing parsing in a faster language than Python might help
obs = env.reset()

In [5]:
# during the reset a problem is chosen at random
print(env.render())

cnf(b_equals_bb, hypothesis, equal_sets(b, bb)).
cnf(element_of_b, hypothesis, member(element_of_b, b)).
cnf(prove_element_of_bb, negated_conjecture, ~member(element_of_b, bb)).
cnf(membership_in_subsets, axiom, ~member(X0, X2) | ~subset(X2, X3) | member(X0, X3)).
cnf(subsets_axiom1, axiom, subset(X2, X3) | member(member_of_1_not_of_2(X2,X3), X2)).
cnf(subsets_axiom2, axiom, ~member(member_of_1_not_of_2(X2,X3), X3) | subset(X2, X3)).
cnf(set_equal_sets_are_subsets1, axiom, ~equal_sets(X2, X3) | subset(X2, X3)).
cnf(set_equal_sets_are_subsets2, axiom, ~equal_sets(X3, X2) | subset(X2, X3)).
cnf(subsets_are_set_equal_sets, axiom, ~subset(X4, X1) | ~subset(X1, X4) | equal_sets(X1, X4)).


In [6]:
# observation is a dict with two keys: "action_mask" and "real_obs"
# "action_mask" is 1 for actions an agent can choose and 0 for the rest
# "real_obs" is a JSON representation of logical clauses
print(obs.keys())
print(obs["action_mask"][:20])
print(obs["real_obs"][0])

dict_keys(['real_obs', 'action_mask'])
[1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
b'{"literals":[{"negated":false,"atom":{"index":1528,"arguments":[{"index":77,"arguments":[]},{"index":1268,"arguments":[]}]}}],"label":"b_equals_bb","role":"hypothesis","inference_parents":[],"inference_rule":null,"processed":false,"birth_step":0}'


In [7]:
 # possible actions are indices of not processed clauses from the state
# formally, action space is a large enough discrete
print(env.action_space)
# but the number of ones in the mask
print(obs["action_mask"].sum())
# equals to number of not processed clauses
print(sum(
    [1.0 for clause in obs["real_obs"] if b'"processed": True' not in clause]
))

Discrete(100000)
9.0
9.0


In [8]:
# only "ansi" and "human" rendering methods are implemented
# "human" (default) returns TPTP representation
# "ansi" returns a JSON string representing the state
print(env.render(mode="ansi")[:1000])

[b'{"literals":[{"negated":false,"atom":{"index":1528,"arguments":[{"index":77,"arguments":[]},{"index":1268,"arguments":[]}]}}],"label":"b_equals_bb","role":"hypothesis","inference_parents":[],"inference_rule":null,"processed":false,"birth_step":0}', b'{"literals":[{"negated":false,"atom":{"index":6,"arguments":[{"index":3000,"arguments":[]},{"index":77,"arguments":[]}]}}],"label":"element_of_b","role":"hypothesis","inference_parents":[],"inference_rule":null,"processed":false,"birth_step":0}', b'{"literals":[{"negated":true,"atom":{"index":6,"arguments":[{"index":3000,"arguments":[]},{"index":1268,"arguments":[]}]}}],"label":"prove_element_of_bb","role":"negated_conjecture","inference_parents":[],"inference_rule":null,"processed":false,"birth_step":0}', b'{"literals":[{"negated":true,"atom":{"index":6,"arguments":[{"index":0},{"index":2}]}},{"negated":true,"atom":{"index":4,"arguments":[{"index":2},{"index":3}]}},{"negated":false,"atom":{"index":6,"arguments":[{"index":0},{"index":3}

In [9]:
# the package includes a simple episode function
from gym_saturation.agent_testing import episode
# and an agent selecting the shortest clause five times
# and then once --- the oldest clause
from gym_saturation.agent_testing import SizeAgeAgent

env.unwrapped.problem_list = [
    files("gym_saturation")
    .joinpath(
        os.path.join("resources", "TPTP-mock", "Problems", "SET", "SET001-1.p")
    )
]
# this agent manages to solve a simple problem from TPTP
episode(env, SizeAgeAgent(5, 1))

1.0

In [10]:
# the proof here consists of only three steps
# but this agent finds it only after 11 steps
# a trained DL agent might have done better
print(env._elapsed_steps)

10


In [11]:
# one can now check the TSTP proof found
print(env.tstp_proof)

cnf(xbb73e95c_ee54_11ec_adc5_5d6e15ba2bdd, lemma, ~subset(b, X3) | member(element_of_b, X3), inference(resolution, [], [element_of_b, membership_in_subsets])).
cnf(xbb72d7c5_ee54_11ec_8f1c_5d6e15ba2bdd, lemma, subset(b, bb), inference(resolution, [], [b_equals_bb, set_equal_sets_are_subsets1])).
cnf(xbb75bd9d_ee54_11ec_b2d3_5d6e15ba2bdd, lemma, ~subset(b, bb), inference(resolution, [], [prove_element_of_bb, xbb73e95c_ee54_11ec_adc5_5d6e15ba2bdd])).
cnf(xbb766fbc_ee54_11ec_bbc8_5d6e15ba2bdd, lemma, $false, inference(resolution, [], [xbb72d7c5_ee54_11ec_8f1c_5d6e15ba2bdd, xbb75bd9d_ee54_11ec_b2d3_5d6e15ba2bdd])).


In [12]:
# paramodulation test
env.unwrapped.problem_list = [
    files("gym_saturation")
    .joinpath(
        os.path.join("resources", "TPTP-mock", "Problems", "TST", "TST002-1.p")
    )
]
env.reset()
print(env.render())
episode(env, SizeAgeAgent(5, 1))
# here the proof consists of five steps
# but the agent needs ten
print(env.tstp_proof)
print(
    len(env.tstp_proof.split("\n")), 
    env._elapsed_steps
)

cnf(a1, hypothesis, q(a)).
cnf(a2, hypothesis, ~q(a) | f(X0) = X0).
cnf(a3, hypothesis, p(X0) | p(f(a))).
cnf(c, negated_conjecture, ~p(X0) | ~p(f(X0))).
cnf(xbcfe3569_ee54_11ec_bda3_5d6e15ba2bdd, lemma, p(f(a)), inference(factoring, [], [a3])).
cnf(xbd007267_ee54_11ec_9f63_5d6e15ba2bdd, lemma, f(X0) = X0, inference(resolution, [], [a1, a2])).
cnf(xbcff5039_ee54_11ec_ac2f_5d6e15ba2bdd, lemma, ~p(a), inference(resolution, [], [xbcfe3569_ee54_11ec_bda3_5d6e15ba2bdd, c])).
cnf(xbd01f98e_ee54_11ec_af26_5d6e15ba2bdd, lemma, p(a), inference(paramodulation, [], [a3, xbd007267_ee54_11ec_9f63_5d6e15ba2bdd])).
cnf(xbd02c703_ee54_11ec_bc5d_5d6e15ba2bdd, lemma, $false, inference(resolution, [], [xbcff5039_ee54_11ec_ac2f_5d6e15ba2bdd, xbd01f98e_ee54_11ec_af26_5d6e15ba2bdd])).
5 9
